In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from pathlib import Path
from ml_assemblr.main_components.data_pod import DataPod
from ml_assemblr.main_components.data_pod_list import DataPodList
from home_credit_helper.config import cfg, HomeCreditRunnerConfigs
from home_credit_helper.constant import *
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [2]:
from ml_assemblr.transfromer.column_type.features_setter import TopDownFeaturesSetter
from ml_assemblr.transfromer.column_type.column_type_setter import ColumnTypeSetter
from ml_assemblr.transfromer.model.xgb_model import XGBModel, get_xgb_config
from home_credit_helper.evaluation.main import evaluation
from home_credit_helper.utils.submission_utils import prepare_submission_files

In [3]:
import xgboost as xgb
import optuna

In [4]:
pd.set_option("display.max_columns", None)

In [5]:
with open(cfg.research_cache_path / "03_dp.pkl", "rb") as f:
    dp: DataPod = pickle.load(f)

In [6]:
selected_features_path = cfg.research_cache_path / "06_selected_features.pkl"
is_full_features = not os.path.exists(selected_features_path)

if not is_full_features:
    with open(selected_features_path, "rb") as f:
        selected_features: list[str] = pickle.load(f)
    
    feature_setters = ColumnTypeSetter(column_type_map={"features": selected_features})
else:
    feature_setters = TopDownFeaturesSetter()

dp = dp.fit_transform(feature_setters)
dp_untrain = dp.copy()

In [7]:
optuna_study_path = f"sqlite:///{cfg.research_cache_path / "optuna_studies.db"}"

study = optuna.load_study(
    study_name="xgb-hyperparam-tuning-supset-features", storage=optuna_study_path
)
xgb_config = get_xgb_config(study)

In [8]:
raise

RuntimeError: No active exception to reraise

In [ ]:
# xgb_config = {
#     "xgb_params": {
#         "learning_rate": 0.02,
#         "max_depth": 8,
#         "subsample": 0.8715623,
#         "min_child_weight": 39.3259775,
#         "colsample_bytree": 0.9497036,
#         "min_split_loss": 0.0222415,
#         "max_leaves": 34,
#         "reg_alpha": 0.041545473,
#         "reg_lambda": 0.0735294,
#         "objective": "binary:logistic",
#     },
#     "num_boost_round": 209,
# }

In [9]:
xgb_model = XGBModel(**xgb_config)
dp = dp.fit_transform(xgb_model)
dp = evaluation(dp, "auroc_test", fast_mode=True)

In [10]:
len(dp.main_column_type.features)

300

In [11]:
dp.variables["evaluation"]["df_evaluation"]

,metric,value
0,auroc_train,0.880225
1,auroc_valid,NaN
2,auroc_test,0.784900


In [12]:
dp.variables["evaluation"]["objective_auroc_test"]

0.7849003131187375

In [13]:
raise

RuntimeError: No active exception to reraise

# Retrain on all available data

In [14]:
dp = dp_untrain.copy()

dp.main_df["SPLIT"] = np.where(dp.main_df["SPLIT"] != "production", "train",dp.main_df["SPLIT"])
xgb_model = XGBModel(**xgb_config)
dp = dp.fit_transform(xgb_model)
dp = evaluation(dp, "auroc_test", fast_mode=True)

In [15]:
dp.variables["evaluation"]["df_evaluation"]

,metric,value
0,auroc_train,0.874047
1,auroc_valid,NaN
2,auroc_test,NaN


In [16]:
dp.variables["evaluation"]["objective_auroc_test"]

nan

In [17]:
raise

RuntimeError: No active exception to reraise

In [18]:
prepare_submission_files(dp, cfg)